In [0]:
from IPython import get_ipython


In [0]:
# ms-python.python added
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'notebooks'))
    print(os.getcwd())
except:
    pass



In [0]:
from IPython import get_ipython


  ## First model
  ### With different hyperparameters

  A factory to create models with different hyperparamters
  should be created and then evaluated to choose the model
  which is most promising

In [0]:
import sys
from os.path import join

sys.path.append('..')

from src.get_directories import get_directories
from src.training_env import reset_and_populate

raw, interim, processed = get_directories()
reset_and_populate(raw, processed, [300, 100, 100])

log_dir = join('..', 'logs')
from src.training_env import reset
reset(log_dir)


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import TensorBoard

# TODO: loss function should be changable... Optimizer too i guess...
class model:
    def __init__(self, image_size, dense_layers, optimizer, training_params):
        self.__epochs=training_params['epochs']
        self.__steps_per_epoch=training_params['steps_per_epoch']
        self.__validation_steps=training_params['validation_steps']

        generator = lambda dir: ImageDataGenerator(rescale=1./255).flow_from_directory(
            dir,
            target_size=(image_size, image_size),
            batch_size=training_params['batch_size'],
            class_mode='binary')

        self.train_generator = generator(join(processed, 'train'))
        self.validation_generator = generator(join(processed, 'validation'))
        self.test_generator = generator(join(processed, 'test'))

        self.model = models.Sequential()
        self.model.add(layers.Flatten(input_shape=(image_size, image_size, 3)))
        for layer in dense_layers:
            self.model.add(layers.Dense(layer, 'relu'))
        self.model.add(layers.Dense(3, 'softmax'))

        self.model.compile(loss=training_params['loss'], optimizer=optimizer, metrics=['acc'])

        # TODO: This callack should have a name for better evaluation of results...
        self.callbacks = [ TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            embeddings_freq=1) ]
    

    def train(self):
        history = self.model.fit_generator(
            self.train_generator,
            epochs=self.__epochs,
            steps_per_epoch=self.__steps_per_epoch,
            validation_data=self.validation_generator,
            validation_steps=self.__validation_steps,
            callbacks=self.callbacks)
        return history

    def evaluate(self):
        return self.model.evaluate_generator(self.test_generator)



In [0]:
from tensorflow.keras.optimizers import SGD

op1 = SGD(
    lr=0.001,
    decay=1e-5,
    momentum=0.9,
    nesterov=False)

tp1 = {
    'loss': 'sparse_categorical_crossentropy',
    'batch_size': 50,
    'epochs': 10,
    'steps_per_epoch': 100,
    'validation_steps': 50
}
m1 = model(image_size=32, dense_layers=[32, 16], optimizer=op1, training_params=tp1)

m1.train()
print(m1.evaluate())



In [0]:
get_ipython().run_line_magic('load_ext', 'tensorboard')
get_ipython().run_line_magic('tensorboard', '--logdir {log_dir}')
